#import library

In [ ]:
import pandas as pd

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
import numpy as np


# Contoh dataset
# text_data = "bagaimana jika contohnya itu banyak sekaligus menjadi contoh yang sesuai contoh dan contoh menjadi contoh lainnya"
df = pd.read_csv('data.csv', encoding='utf-8');

#cek shape dari data

In [ ]:
df.shape

(818, 1)

#Training data model

In [ ]:
from sklearn.model_selection import train_test_split

text_data = df['teks']

# Membagi data menjadi train dan test dengan rasio 80:20
train_data, test_data = train_test_split(text_data, test_size=0.2, random_state=42)

# Tokenisasi pada data latih
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_data)

total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in train_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')
X_train, y_train = input_sequences[:,:-1], input_sequences[:,-1]
y_train = tf.keras.utils.to_categorical(y_train, num_classes=total_words)

# Tokenisasi pada data uji
test_sequences = tokenizer.texts_to_sequences(test_data)
test_input_sequences = []
for line in test_sequences:
    for i in range(1, len(line)):
        n_gram_sequence = line[:i+1]
        test_input_sequences.append(n_gram_sequence)

test_input_sequences = tf.keras.preprocessing.sequence.pad_sequences(test_input_sequences, maxlen=max_sequence_length, padding='pre')
X_test, y_test = test_input_sequences[:,:-1], test_input_sequences[:,-1]
y_test = tf.keras.utils.to_categorical(y_test, num_classes=total_words)

# Membuat model LSTM
model = Sequential()
model.add(Embedding(total_words, 50, input_length=max_sequence_length-1))
model.add(LSTM(512))
model.add(Dense(total_words, activation='softmax'))

# Kompilasi model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Melatih model
model.fit(X_train, y_train, epochs=100, verbose=2, validation_data=(X_test, y_test))


Epoch 1/100
159/159 - 24s - loss: 7.0723 - accuracy: 0.0162 - val_loss: 6.4728 - val_accuracy: 0.0205 - 24s/epoch - 152ms/step
Epoch 2/100
159/159 - 7s - loss: 6.6750 - accuracy: 0.0213 - val_loss: 6.4669 - val_accuracy: 0.0235 - 7s/epoch - 43ms/step
Epoch 3/100
159/159 - 4s - loss: 6.5343 - accuracy: 0.0203 - val_loss: 6.4047 - val_accuracy: 0.0266 - 4s/epoch - 28ms/step
Epoch 4/100
159/159 - 4s - loss: 6.3500 - accuracy: 0.0273 - val_loss: 6.4614 - val_accuracy: 0.0338 - 4s/epoch - 22ms/step
Epoch 5/100
159/159 - 3s - loss: 6.0930 - accuracy: 0.0286 - val_loss: 6.4199 - val_accuracy: 0.0164 - 3s/epoch - 17ms/step
Epoch 6/100
159/159 - 3s - loss: 5.7450 - accuracy: 0.0405 - val_loss: 6.5151 - val_accuracy: 0.0266 - 3s/epoch - 19ms/step
Epoch 7/100
159/159 - 3s - loss: 5.2888 - accuracy: 0.0535 - val_loss: 6.6860 - val_accuracy: 0.0235 - 3s/epoch - 17ms/step
Epoch 8/100
159/159 - 3s - loss: 4.7448 - accuracy: 0.0877 - val_loss: 6.9330 - val_accuracy: 0.0246 - 3s/epoch - 18ms/step
Epoch

#Evaluate data train

In [ ]:
# Evaluasi model pada data train
evaluation_results = model.evaluate(X_train, y_train)

# Menampilkan hasil evaluasi
print("Loss:", evaluation_results[0])
print("Accuracy:", evaluation_results[1])


159/159 [==============================] - 1s 6ms/step - loss: 0.1692 - accuracy: 0.9269
Loss: 0.1692229062318802
Accuracy: 0.9269352555274963


#Testing model setelah di train

In [ ]:
# Fungsi untuk membuat teks generatif tanpa kata berulang
def generate_response(seed_text, next_words, model, max_sequence_length, tokenizer):
    generated_text = seed_text

    for _ in range(next_words):
        # Mengonversi seed_text menjadi token dan melakukan padding
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')

        # Memprediksi kata selanjutnya berdasarkan seed_text
        predicted_probs = model.predict(token_list, verbose=0)[0]

        # Pilih kata dengan probabilitas tertinggi
        predicted_index = np.argmax(predicted_probs)
        predicted_word = tokenizer.index_word.get(predicted_index, "")

        # Menambahkan kata ke generated_text
        generated_text += " " + predicted_word

        # Menggunakan jawaban sistem sebagai seed_text untuk prediksi berikutnya
        seed_text = generated_text

    return generated_text

# Contoh penggunaan model untuk menghasilkan jawaban sistem sebagai output
seed_input = "kak"
generated_response = generate_response(seed_input, 20, model, max_sequence_length, tokenizer)
print(generated_response)

kak soalnya setengahnya lagi ada di mata kasih terus jantan angka dan lain tidur sementara banget yang jaga mereka melanjutkan istriku


#save model data untuk digunakan kembali

In [ ]:
model.save("data_waifu.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
